In [6]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F10 '''
def F10(X):
    f = bn.F10()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 2522.4 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_10_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_10_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [10]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [11]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(20,30):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-20,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 20
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=218121, Thu Apr 16 18:35:37 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.389596366913493e+08 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.347241743135201e+08 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.51020241e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.4499966e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.303406874956067e+08 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   30    570 3.790041257387605e+07 1.1e+00 1.91e+00  2e+00  2e+00 0:03.3
   64   1216 -1.261587914751351e+06 1.1e+00 1.63e+00  2e+00  2e+00 0:07.4
  100   1900 -3.115298364891279e+07 1.2e+00 1.31e+00  1e+00  1e+00 0:11.3
  155   2945 -1.470415626512342e+08 1.2e+00 7.85e-01  8e-01  8e-01 0:17.3
  200   3800 -2.075422573925550e+08 1.2e+00 4.73e-01  5e-01  5e-01 0:22.2
  273   5187 -2.476793718248765e+08 1.2e+00 2.09e-01  2e-01  2e-01 0:30.3
  300   5700 -2.529247469810638e+08 1.2e+00 1.53e-01  2e-01  2e-01 0:33.3
  391   7429 -2.577096024281064e+08 1.2e+00 5.89e-02  6e-02  6e-02 0:43.3
  400   7600 -2.578172490942824e+08 1.2e+00 5.35e-02  5e-02  5e-02 0:44.3
  500   9500 -2.584555354130165e+08 1.2e+00 1.84e-02  2e-02  2e-02 0:55.1
  600  11400 -2.585306389671051e+08 1.2e+00 6.24e-03  6e-03  6e-03 1:05.6
  700  13300 -2.585379000384868e+08 1.2e+00 2.16e-03  2e-03  2e-03 1:15.7
  800  15200 -2.585389850672927e+08 1.2e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.56875172e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=156875172.076549, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=156875172.076549, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteratio

termination on tolfun=1e-11 (Thu Apr 16 18:55:02 2020)
final/bestever f-value = 1.568752e+08 1.568752e+08
incumbent solution: [ 1.64766551 -4.01212278  0.71188925  4.81456106 -4.41775112  0.13078598
  4.17024755  2.29352067 ...]
std deviations: [2.43629074 2.43659614 2.43630207 2.43660111 2.43659703 2.43673856
 2.43669191 2.4365896  ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=122409, Thu Apr 16 18:55:02 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.423283795276440e+08 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.029254201489823e+08 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


...

 8200 155800 -6.383213886125761e+08 5.7e+00 2.44e-09  2e-09  9e-09 15:02.9


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-638321388.612576, sigma=2.37e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=8217)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-638321388.612576, sigma=2.25e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=8235)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-638321388.612576, sigma=2.13e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=8250)
  ')')
/usr

 8300 157700 -6.383213886125762e+08 5.7e+00 2.10e-09  2e-09  8e-09 15:13.9


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-638321388.612576, sigma=2.09e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=8310)
  ')')


 8400 159600 -6.383213886125762e+08 5.7e+00 2.02e-09  2e-09  8e-09 15:25.0
 8500 161500 -6.383213886125762e+08 5.7e+00 1.97e-09  2e-09  7e-09 15:36.0
 8600 163400 -6.383213886125762e+08 5.8e+00 2.08e-09  2e-09  8e-09 15:46.9
 8700 165300 -6.383213886125762e+08 5.8e+00 2.20e-09  2e-09  8e-09 15:57.9
 8800 167200 -6.383213886125762e+08 5.8e+00 2.04e-09  2e-09  8e-09 16:08.9
 8900 169100 -6.383213886125762e+08 5.8e+00 1.99e-09  2e-09  8e-09 16:19.9
 9000 171000 -6.383213886125762e+08 5.8e+00 1.71e-09  1e-09  7e-09 16:30.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 9100 172900 -6.383213886125762e+08 5.8e+00 1.70e-09  1e-09  6e-09 16:41.9
 9200 174800 -6.383213886125762e+08 5.9e+00 1.79e-09  1e-09  7e-09 16:52.9
 9300 176700 -6.383213886125764e+08 5.9e+00 1.92e-09  2e-09  7e-09 17:03.9
 9400 178600 -6.383213886125762e+08 5.9e+00 1.98e-09  2e-09  8e-09 17:14.9
 9500 180500 -6.383213886125762e+08 5.9e+00 2.10e-09  2e-09  8e-09 17:25.9
 9600 182400 -6.383213886125

In [12]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F10_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F10(X_Values.iloc[i,:200])
    SVM_Fun [i] = F10(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F10(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([1.12086624e+08]), count=array([1])) 1.489221311329135
SVM
ModeResult(mode=array([98746170.29152404]), count=array([1])) 38403660.59923558
ELN
ModeResult(mode=array([1.23875042e+08]), count=array([1])) 50773753.63425064
